# __WSI - ćwiczenie 2.__

### __Algorytmy ewolucyjne i genetyczne__


#### __Treść ćwiczenia__

- Celem ćwiczenia jest implementacja algorytmu ewolucyjnego z selekcją ruletkową, krzyżowaniem
jednopunktowym, mutacją gaussowską oraz sukcesją generacyjną. Następnie należy wykorzystać
implementacje do znalezienia rozwiazania dajacego minimalny koszt dla problemu opisanego w sekcji
- Nalezy znalezc zestaw hiperparametrów, który daje stosunkowo dobry wynik, a nastepnie
zbadac wpływ wybranego hiperparametru.

#### __Opis problemu__


Znalezc optymalną lokalizację dla fabryki, jeżeli korzysta ona z 4 zasobów. Zakładamy przy tym,
że:
- lokalizacja fabryki określona jest przez jej współrzedne $(x_1, x_2) \in R^2$,
- dzienne zapotrzebowanie fabryki to:
  1. $20$ jednostek zasobu $z_1$ transportowanego ze współrzednych $(1, 1)$,
  2. $10$ jednostek zasobu $z_2$ transportowanego ze współrzednych $(-0.5, 1)$,
  3. $5$ jednostek zasobu $z_3$ transportowanego ze współrzednych $(-1, -0.5)$,
  4. $10$ jednostek zasobu $z_4$ transportowanego ze współrzednych $(1, -1)$,
- koszt transportu jednostki każdego z zasobów wyliczany to $1−e^{−d_M}$, gdzie $d_M$ - to odległość
Manhattan od lokalizacji fabryki,
- celem optymalizacji jest znalezienie lokalizacji fabryki, która minimalizuje sumaryczny dzienny
koszt dostarczania zasobów do fabryki.

In [107]:
import numpy as np
from plotly import graph_objs as go
from plotly.express.colors import sample_colorscale
import pandas as pd
import math

RNG = np.random.default_rng()

#### __Definicja funkcji straty__

In [108]:
def manhattan(p1, p2):
    return sum([abs(p1[i] - p2[i]) for i in range(len(p1))])

def trans_cost(p1, p2):
    return 1 - math.exp(-manhattan(p1, p2))

def loss(pos):
    return (20 * trans_cost([1, 1], pos) +
            10 * trans_cost([-0.5, 1], pos) +
            5 * trans_cost([-1, -0.5], pos) +
            10 * trans_cost([1, -1], pos))

#### __Algorytm ewolucyjny__

__Selekcja ruletkowa:__ 
- Funkcja prawdopodobieństwa wyboru osobnika $i$: $p_i = \frac{g_i} {\sum\limits_{j=1}^{N}g_j}$
- Dla problemu minimalizacji zamieniam funkcję straty w funkcję zysku $g(x) = f_{max} - f(x)$
- Najgorszy osobnik ma zerowe szanse przetrwania

In [109]:
def get_prob(population):
    min_grades = [loss(x) for x in population]
    max_grade = max(min_grades)
    min_to_max = [max_grade - f for f in min_grades]
    sum_grades = sum(min_to_max)
    return [g/sum_grades for g in min_to_max]

def select_roulette(population, probability_list, output_size):
    return RNG.choice(population, output_size, replace=True, p=probability_list).tolist()

__Krzyowanie jednopunktowe:__

In [110]:
# krzyżwanie może być w tym zadaniu tylko w punkcie środkowym, inaczej nie ma sensu
# implementacja jest jednak uniwersalna
def one_point_cross(vect):
    split = RNG.integers(1, len(vect[0]))
    return [vect[0][:split] + vect[1][split:], 
            vect[1][:split] + vect[0][split:]]

__Mutacja Gaussowska:__

In [111]:
def mutate_gauss(individual, sigma):
    return [x + RNG.normal(0, sigma) for x in individual]

__Sukcesja generacyjna:__
- Zaimplementowana wyłącznie w celu ewentualnej zamiany na inną metodę sukcesji
- Zwraca odpowiedneigo rozmiaru populację 

In [112]:
def replace(x, size):
    return x[:size]

__Algorytm ewolucyjny:__

In [113]:
def evo(pop_size, num_iters, mutation_strength, cross_probability):
    max_r = 1
    population = [RNG.uniform(-max_r, max_r, 2) for _ in range(pop_size)]
    pop_size = len(population)
    iterations = [population]

    best_p = min(population, key=lambda k: loss(k))
    
    i = 0
    while i < num_iters:
        next_pop = []
        probability_list = get_prob(population)
        while len(next_pop) < pop_size:
            mutate = RNG.choice([True, False], 1, replace=True, 
                                p=[cross_probability, 1 - cross_probability])
            if mutate:  
                next_pop += one_point_cross(select_roulette(population, probability_list, 2))
            else:
                next_pop += select_roulette(population, probability_list, 1)
        population = replace([mutate_gauss(x, mutation_strength) for x in next_pop], pop_size)
        iterations.append(population)
        new_best_p = min(population, key=lambda k: loss(k))
        if loss(new_best_p) < loss(best_p):
            best_p = new_best_p
        i += 1
    
    return np.array(iterations), best_p

#### __Analiza działania i wydajności algorytmu__

__Przykład wywołania__

In [114]:
data = evo(50, 200, 0.005, 0.5)
generations = data[0]
result = data[1]
print(result)

[1.000015970197871, 0.9999525100436429]


__Graficzna reprezentacja ewolucji__

In [115]:
X, Y = generations[:,:,0].flatten(), generations[:,:,1].flatten()

layout = go.Layout(width=700, height=500,
                    title='Factory location',
                    xaxis_title='x',
                    yaxis_title='y',
                    plot_bgcolor='DarkSeaGreen')
fig = go.Figure(layout=layout)

colorscale = sample_colorscale('Agsunset', list(np.linspace(0, 1, len(generations))))
colorscale.reverse()
colorscale[-1]='white'
for i, pop in enumerate(generations):
    fig.add_trace(go.Scatter(x=pop[:,0], y=pop[:,1], mode='markers',
                  name=f'population {i}', 
                  marker=dict(size=6,
                  color=colorscale[i])))
fig.show()

__Doświadczenia statystyczne__

In [116]:
ITERATION_NUMBER = 200
POPULATION_SIZE = 50
MUTATION_STRENGTH = 0.005
CROSSOVER_PROBABILITY = 0.5

test = np.arange(0.005, 0.015, 0.005)
test = [np.round(t, 3) for t in test]
param_name = 'mutation strength'
runs_amount = 10

data = [([evo(POPULATION_SIZE, ITERATION_NUMBER, metric, CROSSOVER_PROBABILITY)  for _ in range(runs_amount)], metric) for metric in test]

col_names = ['x', 'y', 'loss', 'x err (abs)', 'y err (abs)', 'x stddev', 'y stddev']
df_columns = []
df_data = [[]]*runs_amount
for param_data, param_value in data:
    df_data = np.append(df_data, [[x[1][0], x[1][1], loss(x[1]), abs(x[1][0] - 1), abs(x[1][1] - 1), np.std(x[0][-1][:][0]), np.std(x[0][-1][:][1])] for x in param_data], axis=1)
    df_columns += [(f'{param_name} = {param_value}', name) for name in col_names]

df = pd.DataFrame(columns=df_columns, data=df_data)
df.columns = pd.MultiIndex.from_tuples(df.columns)
df


mutation strength = 0.005                                               \
                          x         y       loss x err (abs) y err (abs)   
0                  1.000048  1.000011  21.265763    0.000048    0.000011   
1                  1.000018  0.999907  21.266697    0.000018    0.000093   
2                  1.000065  0.999983  21.266239    0.000065    0.000017   
3                  0.999903  1.000029  21.266876    0.000097    0.000029   
4                  1.000062  0.999933  21.267220    0.000062    0.000067   
5                  1.000074  0.999911  21.267965    0.000074    0.000089   
6                  0.999908  1.000045  21.267155    0.000092    0.000045   
7                  1.000028  0.999957  21.265916    0.000028    0.000043   
8                  1.000198  1.000012  21.269346    0.000198    0.000012   
9                  0.999864  0.999956  21.267848    0.000136    0.000044   

                      mutation strength = 0.01                       \
   x stddev  y stddev                        x         y       loss   
0  0.005360  0.003696                 1.000019  1.000081  21.266739   
1  0.002353  0.002620                 0.999819  0.999910  21.269661   
2  0.002697  0.004083                 0.999627  0.999980  21.271842   
3  0.003740  0.001067                 1.000122  0.999783  21.271756   
4  0.003259  0.012758                 0.999839  1.000118  21.270208   
5  0.013357  0.001842                 0.999841  1.000132  21.270491   
6  0.004992  0.005302                 0.999849  1.000253  21.273225   
7  0.003809  0.006230                 1.000021  1.000076  21.266679   
8  0.006116  0.002812                 1.000007  0.999849  21.267662   
9  0.004518  0.007756                 0.999847  1.000222  21.272524   

                                               
  x err (abs) y err (abs)  x stddev  y stddev  
0    0.000019    0.000081  0.005843  0.001174  
1    0.000181    0.000090  0.014082  0.006016  
2    0.000373    0.000020  0.007129  0.015254  
3    0.000122    0.000217  0.008061  0.002423  
4    0.000161    0.000118  0.017213  0.006467  
5    0.000159    0.000132  0.014458  0.005284  
6    0.000151    0.000253  0.008453  0.026085  
7    0.000021    0.000076  0.004617  0.004296  
8    0.000007    0.000151  0.011502  0.000963  
9    0.000153    0.000222  0.002350  0.011483

In [117]:
df.describe()

mutation strength = 0.005                                                \
                              x          y       loss x err (abs) y err (abs)   
count                 10.000000  10.000000  10.000000   10.000000   10.000000   
mean                   1.000017   0.999974  21.267102    0.000082    0.000045   
std                    0.000100   0.000049   0.001081    0.000053    0.000030   
min                    0.999864   0.999907  21.265763    0.000018    0.000011   
25%                    0.999936   0.999939  21.266353    0.000051    0.000020   
50%                    1.000038   0.999970  21.267016    0.000069    0.000044   
75%                    1.000064   1.000012  21.267691    0.000095    0.000062   
max                    1.000198   1.000045  21.269346    0.000198    0.000093   

                            mutation strength = 0.01                        \
        x stddev   y stddev                        x          y       loss   
count  10.000000  10.000000                10.000000  10.000000  10.000000   
mean    0.005020   0.004817                 0.999899   1.000040  21.270079   
std     0.003159   0.003456                 0.000143   0.000156   0.002374   
min     0.002353   0.001067                 0.999627   0.999783  21.266679   
25%     0.003379   0.002668                 0.999840   0.999927  21.268162   
50%     0.004164   0.003890                 0.999848   1.000079  21.270349   
75%     0.005268   0.005998                 1.000016   1.000128  21.271820   
max     0.013357   0.012758                 1.000122   1.000253  21.273225   

                                                     
      x err (abs) y err (abs)   x stddev   y stddev  
count   10.000000   10.000000  10.000000  10.000000  
mean     0.000135    0.000136   0.009371   0.007945  
std      0.000107    0.000075   0.004785   0.007792  
min      0.000007    0.000020   0.002350   0.000963  
25%      0.000047    0.000083   0.006165   0.002891  
50%      0.000152    0.000125   0.008257   0.005650  
75%      0.000160    0.000200   0.013437   0.010229  
max      0.000373    0.000253   0.017213   0.026085

- Mean sum of errors: $ME = \frac{\sum\limits_{i=1}^{n}|x_i - x_{est}|+|y_i - y_{est}|} {2n}$
- Mean sum of deviations: $MD = \frac{\sum\limits_{i=1}^{n}x_i{dev}+y_i{dev}} {2n}$

In [119]:
df_stat = pd.DataFrame()
for param_name in df.columns.get_level_values(0).unique():
    df_stat[(param_name, 'ME')] = [df.loc[:, [(param_name, 'x err (abs)'), (param_name, 'y err (abs)')]].sum(axis=1).mean()/2]
    df_stat[(param_name, 'MD')] = [df.loc[:, [(param_name, 'x stddev'), (param_name, 'y stddev')]].sum(axis=1).mean()/2]
df_stat.columns = pd.MultiIndex.from_tuples(df_stat.columns)
df_stat

mutation strength = 0.005           mutation strength = 0.01          
                         ME        MD                       ME        MD
0                  0.000063  0.004918                 0.000135  0.008658